In [1]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup as bs

headers = {'Accept-Language': 'en-US,en;q=0.9'}  # to make sure the data will not auto-translate

pages = range(1, 11) # the range of pages to scrap, remember: last value of range is exclusive
                     # not necessary for one page scrapping

In [2]:
 # where the data will be stored before the creation of the Dataframe

movie_name = []
year = []
duration = []
rating = []
imdb_score = []
metascore = []
votes = []
gross = []
poster = []
description = []

In [3]:
for i in pages:
    
    # use the url of the second page, remove the number at the end and apply the range
    url = 'https://www.imdb.com/list/*****/?sort=list_order,asc&st_dt=&mode=detail&page=' + str(i)
    response = requests.get(url, headers=headers)
    soup = bs(response.content, 'html.parser')
    
    movie_data = soup.find_all('div', {'class': 'lister-item mode-detail'})  # the repeated container of data from the page
    
    for item in movie_data:
        
        m_name = item.h3.a.text
        movie_name.append(m_name) # title

        m_year = item.h3.find('span', {'class': 'lister-item-year text-muted unbold'}).text.replace('(', '').replace(')', '').replace(' TV Movie', '').replace('I ', '')
        year.append(m_year) # release year

        m_duration = item.p.find('span', {'class': 'runtime'}).text.replace(' min', '')
        duration.append(m_duration) # movie duration

        m_rating = item.p.find('span', {'class': 'certificate'}).text if item.p.find('span', {'class': 'certificate'}) else 'none'
        rating.append(m_rating) # movie classification/rating

        m_imdb_score = item.find('span', {'class': 'ipl-rating-star__rating'}).text if item.find('span', {'class': 'ipl-rating-star__rating'}) else '0'
        imdb_score.append(m_imdb_score) # imdb public score

        m_metascore = item.find_next('div', {'class': 'inline-block ratings-metascore'}).find('span').text.replace(' ', '') if item.find('div', {'class': 'inline-block ratings-metascore'}) else '0'
        metascore.append(m_metascore) # critic score from Metascore

        m_votes = item.find_all('span', {'name': 'nv'})[0].text
        votes.append(m_votes) # amount of votes for public imdb

        m_gross = item.find_all('span', {'name': 'nv'})[-1]['data-value'] if len(item.find_all('span', {'name': 'nv'})) > 1 else '0'
        gross.append(m_gross) # movie total gross

        m_poster = item.find('div', {'class': 'lister-item-image ribbonize'}).find('img')['loadlate']
        poster.append(m_poster) # posters url

        m_description = item.find('div', {'class': 'lister-item-content'}).find('p', {'class': ''}).text.replace('\n', '').replace(';', ',')
        description.append(m_description) # movie summary/description

In [4]:
 # Dataframe creation
movies_df = pd.DataFrame({'Title': movie_name, 'Year': year, 'Duration': duration, 'Rating': rating, 'IMDB Score': imdb_score, 'Metascore': metascore, 'Votes': votes, 'Gross': gross, 'Description': description, 'Poster': poster})

In [5]:
movies_df.head()

,Title,Year,Duration,Rating,IMDB Score,Metascore,Votes,Gross,Description,Poster
0,Equilibrium,2002,107,R,7.4,33,"323,672","1,203,974",In an oppressive future where all forms of fee...,https://m.media-amazon.com/images/M/MV5BMTkzMz...
1,Spider-Man 3,2007,139,PG-13,6.3,59,"566,658","336,530,303",A strange black entity from another world bond...,https://m.media-amazon.com/images/M/MV5BYTk3MD...
2,Transformers: Revenge of the Fallen,2009,149,PG-13,6,35,"400,917","402,111,870",Sam Witwicky leaves the Autobots behind for a ...,https://m.media-amazon.com/images/M/MV5BNjk4OT...
3,The Matrix Reloaded,2003,138,R,7.2,62,"581,522","281,492,479","Freedom fighters Neo, Trinity and Morpheus con...",https://m.media-amazon.com/images/M/MV5BODE0Mz...
4,Jumper,2008,88,PG-13,6.1,35,"301,114","80,172,128",A teenager with teleportation abilities sudden...,https://m.media-amazon.com/images/M/MV5BMjEwOT...


In [6]:
 # movies_df.to_csv('name_of_the_dataset.csv')  # saving a file with the data